## Before you read or execute this file:
- Please do not ask me for the logic or debugging, there are some areas where only chatgpt and god knows what happened
- The presentation and my mental state degrade as you keep scrolling down
- This is still the base, can and will be optimised in the future

## Fetching info of the stock
I used TataPower and PowerGrid, change it to your desired stock

In [54]:
import yfinance as yf
import pandas as pd

def fetch_stock_info(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    
    stock_data = {
        "Ticker": ticker,
        "Company Name": info.get("longName", "N/A"),
        "Current Price": info.get("currentPrice", "N/A"),
        "Market Cap": info.get("marketCap", "N/A"),
        "Sector": info.get("sector", "N/A"),
        "Industry": info.get("industry", "N/A"),
        "52-Week High": info.get("fiftyTwoWeekHigh", "N/A"),
        "52-Week Low": info.get("fiftyTwoWeekLow", "N/A"),
        "P/E Ratio": info.get("trailingPE", "N/A"),
        "Dividend Yield": info.get("dividendYield", "N/A"),
    }
    
    return stock_data

def fetch_historical_prices(ticker, start="2014-01-01", end="2024-01-01"):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end)  # Fetch last 10 years of data
    hist = hist[['Close']].reset_index()  # Keep only Date and Close Price
    hist['Returns'] = hist['Close'].pct_change()  # Calculate daily returns
    hist.dropna(inplace=True)  # Remove first row with NaN return
    return hist

# Example Usage
tickers = ["TATAPOWER.NS", "POWERGRID.NS"]  # Sample tickers

# Fetch fundamental data
portfolio_data = [fetch_stock_info(ticker) for ticker in tickers]
df_portfolio = pd.DataFrame(portfolio_data)

# Fetch historical data for each stock separately
stock_dataframes = {ticker: fetch_historical_prices(ticker) for ticker in tickers}

# Display results
print("Portfolio Information:\n", df_portfolio)

for ticker, df in stock_dataframes.items():
    print(f"\nHistorical Data for {ticker}:\n", df.tail())


Portfolio Information:
          Ticker                             Company Name  Current Price  \
0  TATAPOWER.NS           The Tata Power Company Limited         354.60   
1  POWERGRID.NS  Power Grid Corporation of India Limited         259.15   

      Market Cap     Sector                                 Industry  \
0  1133702283264  Utilities  Utilities - Independent Power Producers   
1  2410250502144  Utilities           Utilities - Regulated Electric   

   52-Week High  52-Week Low  P/E Ratio  Dividend Yield  
0        494.85       326.35  29.723387            0.56  
1        366.25       252.20  15.555222            5.73  

Historical Data for TATAPOWER.NS:
                           Date       Close   Returns
2460 2023-12-22 00:00:00+05:30  325.144165 -0.005783
2461 2023-12-26 00:00:00+05:30  323.899933 -0.003827
2462 2023-12-27 00:00:00+05:30  323.352478 -0.001690
2463 2023-12-28 00:00:00+05:30  328.279663  0.015238
2464 2023-12-29 00:00:00+05:30  330.618835  0.007126

Hist

#### Each stock dataframe is in a dictionary and can be accessed individually like:
```
stock_dataframes["TATAPOWER.NS"]
```

In [55]:
import yfinance as yf
import pandas as pd

def fetch_stock_info(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    
    stock_data = {
        "Ticker": ticker,
        "Company Name": info.get("longName", "N/A"),
        "Current Price": info.get("currentPrice", "N/A"),
        "Market Cap": info.get("marketCap", "N/A"),
        "Sector": info.get("sector", "N/A"),
        "Industry": info.get("industry", "N/A"),
        "52-Week High": info.get("fiftyTwoWeekHigh", "N/A"),
        "52-Week Low": info.get("fiftyTwoWeekLow", "N/A"),
        "P/E Ratio": info.get("trailingPE", "N/A"),
        "Dividend Yield": info.get("dividendYield", "N/A"),
    }
    
    return stock_data

def fetch_historical_prices(ticker, start="2014-01-01", end="2024-01-01"):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end)[['Close']].reset_index()
    hist['Returns'] = hist['Close'].pct_change()
    hist.dropna(inplace=True)
    return hist

# Example Usage
tickers = ["TATAPOWER.NS", "POWERGRID.NS"]

# Fetch fundamental data
df_portfolio = pd.DataFrame([fetch_stock_info(ticker) for ticker in tickers])

# Fetch historical data and store each in a separate DataFrame
stock_dataframes = {}
for ticker in tickers:
    stock_dataframes[ticker] = fetch_historical_prices(ticker)
    globals()[f"df_{ticker.replace('.', '_')}"] = stock_dataframes[ticker]

# Display results
print("Portfolio Information:\n", df_portfolio)

for ticker, df in stock_dataframes.items():
    print(f"\nHistorical Data for {ticker}:\n", df.tail())


Portfolio Information:
          Ticker                             Company Name  Current Price  \
0  TATAPOWER.NS           The Tata Power Company Limited         354.60   
1  POWERGRID.NS  Power Grid Corporation of India Limited         259.15   

      Market Cap     Sector                                 Industry  \
0  1133702283264  Utilities  Utilities - Independent Power Producers   
1  2410250502144  Utilities           Utilities - Regulated Electric   

   52-Week High  52-Week Low  P/E Ratio  Dividend Yield  
0        494.85       326.35  29.723387            0.56  
1        366.25       252.20  15.555222            5.73  

Historical Data for TATAPOWER.NS:
                           Date       Close   Returns
2460 2023-12-22 00:00:00+05:30  325.144165 -0.005783
2461 2023-12-26 00:00:00+05:30  323.899933 -0.003827
2462 2023-12-27 00:00:00+05:30  323.352478 -0.001690
2463 2023-12-28 00:00:00+05:30  328.279663  0.015238
2464 2023-12-29 00:00:00+05:30  330.618835  0.007126

Hist

### Tried implementing front end

In [56]:
pip install streamlit


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [57]:
import streamlit as st
import yfinance as yf
import pandas as pd

# Function to fetch stock details
def fetch_stock_info(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    
    return {
        "Ticker": ticker,
        "Company Name": info.get("longName", "N/A"),
        "Current Price": info.get("currentPrice", "N/A"),
        "Market Cap": info.get("marketCap", "N/A"),
        "Sector": info.get("sector", "N/A"),
        "Industry": info.get("industry", "N/A"),
        "52-Week High": info.get("fiftyTwoWeekHigh", "N/A"),
        "52-Week Low": info.get("fiftyTwoWeekLow", "N/A"),
        "P/E Ratio": info.get("trailingPE", "N/A"),
        "Dividend Yield": info.get("dividendYield", "N/A"),
    }

# Streamlit UI
st.title("Portfolio Stock Information")

# User Input for Stock Tickers
tickers_input = st.text_input("Enter Stock Tickers (comma-separated)", "TATAPOWER.NS, POWERGRID.NS")

if st.button("Fetch Data"):
    tickers = [t.strip() for t in tickers_input.split(",")]  # Split and clean input
    portfolio_data = [fetch_stock_info(ticker) for ticker in tickers]

    df_portfolio = pd.DataFrame(portfolio_data)  # Convert to DataFrame
    st.write(df_portfolio)  # Display the table


2025-02-25 02:00:14.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:00:14.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:00:14.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:00:14.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:00:14.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:00:14.698 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:00:14.699 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:00:14.699 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Ran this command on terminal to host it locally, but failed. Let's leave it for now and focus on mapping.
```
streamlit run /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel_launcher.py 
```

Will try and integrate it with frontend later.

Also streamlit requires the script to be running at all times, so not the best option.

---
## Sharpe Ratio


### Calculation
```
annual_return = mean_daily_return * 252 
```
Should we use this formula, or calculate annual return of each year and then take the average. idk if it makes a difference, but for now I have chosen this.

Also the ```risk_free_rate```: Annual risk-free rate is taken to be default 5%. Can be changed if needed.

In [58]:
import numpy as np

def calculate_sharpe_ratio(df, risk_free_rate=0.05):
    """
    Calculates the Sharpe Ratio for a given stock's historical data.
    
    df: DataFrame with 'Returns' column
    risk_free_rate: Annual risk-free rate (default 5%)
    
    Returns: Sharpe Ratio
    """
    mean_daily_return = df['Returns'].mean()
    std_dev = df['Returns'].std()
    
    # Convert risk-free rate to daily (Assuming 252 trading days)
    daily_risk_free_rate = risk_free_rate / 252
    
    # Annualized return and volatility
    annual_return = mean_daily_return * 252   # SHOULD WE USE IT LIKE THIS, OR GENERATE ACTUAL ANNUAL RETURN FOR EACH YEAR?
    print(annual_return)
    annual_volatility = std_dev * np.sqrt(252)
    
    # Sharpe Ratio calculation
    sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility
    return sharpe_ratio

# Calculate Sharpe Ratio for each stock
sharpe_ratios = {ticker: calculate_sharpe_ratio(df) for ticker, df in stock_dataframes.items()}

# Display results
for ticker, sharpe in sharpe_ratios.items():
    print(f"Sharpe Ratio for {ticker}: {sharpe:.2f}")


0.21925873346308927
0.21879429764616343
Sharpe Ratio for TATAPOWER.NS: 0.48
Sharpe Ratio for POWERGRID.NS: 0.69


### Rating

In [59]:
import numpy as np


def assign_sharpe_rating(sharpe_ratio):
    """
    Assigns a rating and score based on the Sharpe Ratio thresholds.
    """
    if sharpe_ratio >= 2.5:
        return "AAA", 8
    elif 2.0 <= sharpe_ratio < 2.5:
        return "AA", 7
    elif 1.5 <= sharpe_ratio < 2.0:
        return "A", 6
    elif 1.0 <= sharpe_ratio < 1.5:
        return "BBB", 5
    elif 0.5 <= sharpe_ratio < 1.0:
        return "BB", 4
    elif 0.0 <= sharpe_ratio < 0.5:
        return "B", 3
    elif -0.5 <= sharpe_ratio < 0.0:
        return "C", 2
    else:
        return "D", 1  # Sharpe Ratio < -0.5

# Calculate Sharpe Ratio and assign ratings for each stock
sharpe_ratings = {}
for ticker, df in stock_dataframes.items():
    sharpe_ratio = calculate_sharpe_ratio(df)
    rating, score = assign_sharpe_rating(sharpe_ratio)
    sharpe_ratings[ticker] = {"Sharpe Ratio": sharpe_ratio, "Rating": rating, "Score": score}

# Convert results to a DataFrame
df_sharpe = pd.DataFrame.from_dict(sharpe_ratings, orient="index")

# Display results
print(df_sharpe)


0.21925873346308927
0.21879429764616343
              Sharpe Ratio Rating  Score
TATAPOWER.NS      0.479359      B      3
POWERGRID.NS      0.693314     BB      4


---
## Sortino Ratio

### Calculation

Again same question as Sharpe ratio regarding the ```annualized_returns```.

The ```risk_free_rate```: Annual risk-free rate is taken to be default 3%.

Someone please standardise them in the codes below.

In [60]:
annual_returns = {}  # Dictionary to store annualized returns

def calculate_sortino_ratio(ticker, stock_returns, risk_free_rate=0.03):
    """
    Calculates the Sortino Ratio for a given stock's returns and stores annualized return.
    :param ticker: Stock ticker symbol
    :param stock_returns: Series of daily returns
    :param risk_free_rate: Annual risk-free rate (default 3%)
    :return: Sortino Ratio
    """
    # Calculate annualized return and store it
    annualized_return = np.mean(stock_returns) * 252  
    annual_returns[ticker] = annualized_return  # Store for Calmar Ratio

    downside_returns = stock_returns[stock_returns < 0]  
    downside_deviation = np.std(downside_returns) * np.sqrt(252)  
    
    if downside_deviation == 0:
        return np.nan

    sortino_ratio = (annualized_return - risk_free_rate) / downside_deviation
    return sortino_ratio

# Compute Sortino Ratio for each stock and store annual returns
sortino_ratios = {ticker: calculate_sortino_ratio(ticker, df["Returns"]) for ticker, df in stock_dataframes.items()}

# Display results
for ticker, sortino in sortino_ratios.items():
    print(f"Sortino Ratio for {ticker}: {sortino:.2f}")

Sortino Ratio for TATAPOWER.NS: 0.81
Sortino Ratio for POWERGRID.NS: 1.20


### Ranking

In [61]:
def assign_sortino_rating(sortino_ratio):
    """Assigns a rating based on the Sortino Ratio thresholds."""
    if sortino_ratio >= 3.0:
        return "AAA", 8
    elif 2.5 <= sortino_ratio < 3.0:
        return "AA", 7
    elif 2.0 <= sortino_ratio < 2.5:
        return "A", 6
    elif 1.5 <= sortino_ratio < 2.0:
        return "BBB", 5
    elif 1.0 <= sortino_ratio < 1.5:
        return "BB", 4
    elif 0.5 <= sortino_ratio < 1.0:
        return "B", 3
    elif 0.0 <= sortino_ratio < 0.5:
        return "C", 2
    else:
        return "D", 1  # Sortino Ratio < 0.0

# Compute Sortino Ratio and Assign Ratings
sortino_ratings = {}
for ticker, df in stock_dataframes.items():
    sortino_ratio = calculate_sortino_ratio(ticker,df["Returns"])
    rating, score = assign_sortino_rating(sortino_ratio)
    sortino_ratings[ticker] = {"Sortino Ratio": sortino_ratio, "Rating": rating, "Score": score}

# Convert to DataFrame for better visualization
df_sortino = pd.DataFrame.from_dict(sortino_ratings, orient="index")
print(df_sortino)


              Sortino Ratio Rating  Score
TATAPOWER.NS       0.810150      B      3
POWERGRID.NS       1.197618     BB      4


---
## Maximum Drawdown

### Calculation

In [62]:
import numpy as np

def calculate_max_drawdown(df):
    """
    Calculates the Maximum Drawdown (MDD) for a given stock's historical data.
    
    df: DataFrame with 'Close' price column
    
    Returns: Maximum Drawdown (as a percentage)
    """
    # Calculate cumulative max of closing prices (Peak values)
    df['Peak'] = df['Close'].cummax()
    
    # Calculate drawdown (percentage decline from peak)
    df['Drawdown'] = ((df['Peak'] - df['Close']) / df['Peak'] ) 
    
    # Find the maximum drawdown
    max_drawdown = df['Drawdown'].max()
    
    return max_drawdown

# Calculate Max Drawdown for each stock
max_drawdowns = {ticker: calculate_max_drawdown(df) for ticker, df in stock_dataframes.items()}

# Convert results to a DataFrame
df_max_drawdown = pd.DataFrame.from_dict(max_drawdowns, orient="index", columns=["Max Drawdown"])

# Display results
print(df_max_drawdown)


              Max Drawdown
TATAPOWER.NS      0.724747
POWERGRID.NS      0.293243


## Ranking

In [63]:
def rank_max_drawdown(mdd):
    """
    Assigns a rating and score based on the maximum drawdown (MDD) value.
    
    mdd: Maximum Drawdown (as a decimal, e.g., 0.20 for 20%)
    
    Returns: (Rating, Score)
    """
    if mdd < 0.10:
        return "AAA", 8
    elif 0.10 <= mdd < 0.15:
        return "AA", 7
    elif 0.15 <= mdd < 0.20:
        return "A", 6
    elif 0.20 <= mdd < 0.25:
        return "BBB", 5
    elif 0.25 <= mdd < 0.30:
        return "BB", 4
    elif 0.30 <= mdd < 0.35:
        return "B", 3
    elif 0.35 <= mdd < 0.40:
        return "C", 2
    else:
        return "D", 1

# Apply ranking to each stock's Max Drawdown
df_max_drawdown[["Rating", "Score"]] = df_max_drawdown["Max Drawdown"].apply(lambda x: pd.Series(rank_max_drawdown(x)))

# Display results
print(df_max_drawdown)


              Max Drawdown Rating  Score
TATAPOWER.NS      0.724747      D      1
POWERGRID.NS      0.293243     BB      4


---
## Calmar Ratio

### Calculation

In [64]:
def calculate_calmar_ratio(ticker):
    """
    Calculates the Calmar Ratio for a given stock using its annual return and max drawdown.
    
    :param ticker: Stock ticker symbol
    :return: Calmar Ratio
    """
    # Get the annual return from stored values
    annual_return = annual_returns.get(ticker, np.nan)
    
    # Get the max drawdown from stored values
    max_drawdown = max_drawdowns.get(ticker, np.nan)
    
    if np.isnan(annual_return) or np.isnan(max_drawdown) or max_drawdown == 0:
        return np.nan  # To avoid division by zero

    calmar_ratio = annual_return / max_drawdown
    return calmar_ratio

# Compute Calmar Ratio for each stock
calmar_ratios = {ticker: calculate_calmar_ratio(ticker) for ticker in stock_dataframes.keys()}

# Display results
for ticker, calmar in calmar_ratios.items():
    print(f"Calmar Ratio for {ticker}: {calmar:.2f}")


Calmar Ratio for TATAPOWER.NS: 0.30
Calmar Ratio for POWERGRID.NS: 0.75


## Ranking

In [65]:
import pandas as pd

def calculate_calmar_ratio(ticker, max_drawdowns):
    """
    Calculates the Calmar Ratio for a given stock.
    :param ticker: Stock ticker symbol
    :param max_drawdowns: Dictionary with max drawdowns per ticker
    :return: Calmar Ratio, Rating, Score
    """
    if ticker not in annual_returns or ticker not in max_drawdowns:
        return np.nan, "N/A", np.nan  # Return NaN if data is missing

    annual_return = annual_returns[ticker]
    max_drawdown = max_drawdowns[ticker]

    if max_drawdown == 0:  # Avoid division by zero
        return np.nan, "N/A", np.nan

    calmar_ratio = annual_return / abs(max_drawdown)  # Use absolute value of drawdown

    # Assign Rating & Score
    if calmar_ratio >= 4.0:
        rating, score = "AAA", 8
    elif 3.0 <= calmar_ratio < 4.0:
        rating, score = "AA", 7
    elif 2.0 <= calmar_ratio < 3.0:
        rating, score = "A", 6
    elif 1.5 <= calmar_ratio < 2.0:
        rating, score = "BBB", 5
    elif 1.0 <= calmar_ratio < 1.5:
        rating, score = "BB", 4
    elif 0.5 <= calmar_ratio < 1.0:
        rating, score = "B", 3
    elif 0.0 <= calmar_ratio < 0.5:
        rating, score = "C", 2
    else:
        rating, score = "D", 1

    return calmar_ratio, rating, score


# Compute Calmar Ratio for each stock
calmar_data = {
    "Ticker": [],
    "Calmar Ratio": [],
    "Rating": [],
    "Score": []
}

for ticker in stock_dataframes.keys():
    calmar, rating, score = calculate_calmar_ratio(ticker, max_drawdowns)
    calmar_data["Ticker"].append(ticker)
    calmar_data["Calmar Ratio"].append(calmar)
    calmar_data["Rating"].append(rating)
    calmar_data["Score"].append(score)

# Convert to DataFrame
df_calmar = pd.DataFrame(calmar_data)


# Display DataFrame without default index
print(df_calmar.to_string(index=False))



      Ticker  Calmar Ratio Rating  Score
TATAPOWER.NS      0.302531      C      2
POWERGRID.NS      0.746120      B      3


---
#### Guess who forgot pyfolio exists 🤡
---
## Pyfolio

In [66]:
pip install pyfolio



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Need to downgrade numpy and pandas for pyfolio to work properly

In [67]:
pip install numpy==1.26



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [68]:
import pyfolio as pf
import matplotlib.pyplot as plt

# Store extracted performance metrics
performance_metrics = []

for ticker, df in stock_dataframes.items():
    returns_series = df.set_index("Date")["Returns"]
    
    # Generate Pyfolio Tear Sheet (commented to avoid interactive pop-ups)
    # pf.create_full_tear_sheet(returns_series)
    
    # Extract key metrics
    perf_stats = pf.timeseries.perf_stats(returns_series)
    
    # Store results
    metrics = {
        "Ticker": ticker,
        "Annual Return": perf_stats["Annual return"],
        "Sharpe Ratio": perf_stats["Sharpe ratio"],
        "Sortino Ratio": perf_stats["Sortino ratio"],
        "Max Drawdown": perf_stats["Max drawdown"],
        "Calmar Ratio": perf_stats["Calmar ratio"],
    }
    
    performance_metrics.append(metrics)

# Create DataFrame with extracted performance metrics
df_performance = pd.DataFrame(performance_metrics)

# Display results
print(df_performance)


         Ticker  Annual Return  Sharpe Ratio  Sortino Ratio  Max Drawdown  \
0  TATAPOWER.NS       0.170234      0.620965       0.935197     -0.724747   
1  POWERGRID.NS       0.208184      0.898687       1.356287     -0.293243   

   Calmar Ratio  
0      0.234887  
1      0.709939  


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


### Different result because of different assumptions. I hope....

In [82]:
import numpy as np
import pandas as pd
import yfinance as yf

# Define benchmark index
benchmark_ticker = "^NSEI"

# Fetch benchmark historical data
def fetch_benchmark_data(ticker, start="2014-01-01", end="2024-01-01"):
    benchmark_data = yf.download(ticker, start=start, end=end)

    # Check if data is available
    if benchmark_data.empty:
        raise ValueError(f"Failed to fetch data for {ticker}. Please check the ticker symbol.")

    # Reset index to ensure "Date" is a column
    benchmark_data.reset_index(inplace=True)

    # Flatten MultiIndex columns if present
    benchmark_data.columns = [col[0] if isinstance(col, tuple) else col for col in benchmark_data.columns]

    # Use "Adj Close" if "Close" is missing
    if "Close" not in benchmark_data.columns:
        if "Adj Close" in benchmark_data.columns:
            benchmark_data.rename(columns={"Adj Close": "Close"}, inplace=True)
        else:
            raise KeyError("Neither 'Close' nor 'Adj Close' found in benchmark data.")

    # Calculate benchmark returns
    benchmark_data["Benchmark_Return"] = benchmark_data["Close"].pct_change()
    benchmark_data = benchmark_data[["Date", "Benchmark_Return"]].dropna()  # Keep only necessary columns
    
    return benchmark_data

# Fetch NSEI benchmark data
benchmark_data = fetch_benchmark_data(benchmark_ticker)

# Store additional performance metrics
additional_metrics = []

# Define risk-free rate (assumed 3% annualized)
risk_free_rate = 0.03 / 252  # Convert annual to daily

# Define threshold return for Omega Ratio (assumed 0%)
threshold_return = 0.0

for ticker, df in stock_dataframes.items():
    df = df.copy()
    
    # Ensure 'Date' is a normal column
    if "Date" not in df.columns:
        df.reset_index(inplace=True)
    
    df.set_index("Date", inplace=True)
    
    # Ensure "Returns" column exists
    if "Close" in df.columns:
        df["Returns"] = df["Close"].pct_change()
    
    if "Returns" not in df.columns:
        print(f"Skipping {ticker} due to missing 'Returns' column.")
        continue
    
    # Align benchmark returns with stock returns
    # Convert both dataframes to timezone-naive
    df.index = df.index.tz_localize(None)
    benchmark_data["Date"] = benchmark_data["Date"].dt.tz_localize(None)

    # Align benchmark returns with stock returns
    df = df.join(benchmark_data.set_index("Date"), how="inner")

    # print(df.head(10))  # Check first few rows
    # print(df.describe())  # Check summary stats
    # print(df.isna().sum())  # Check missing values

    
    # Check if data is sufficient
    if "Benchmark_Return" not in df.columns:
        print(f"Skipping {ticker} due to missing benchmark returns.")
        continue
    
    # Define stock and benchmark returns
    Rp = df["Returns"]
    Rb = df["Benchmark_Return"]
    
    # Calculate Beta
    # beta = np.cov(Rp, Rb)[0, 1] / np.var(Rb) if np.var(Rb) != 0 else np.nan
    from scipy.stats import linregress
    beta, alpha, _, _, _ = linregress(Rb, Rp)

    # Calculate Alpha (Annualized)
    annualized_alpha = (Rp.mean() * 252) - (risk_free_rate * 252 + beta * ((Rb.mean() * 252) - (risk_free_rate * 252)))
    
    # Calculate Treynor Ratio
    treynor_ratio = ((Rp.mean() * 252) - (risk_free_rate * 252)) / beta if beta != 0 else np.nan
    
    # Calculate Tracking Error (Standard deviation of excess returns)
    tracking_error = np.std(Rp - Rb) * np.sqrt(252)
    
    # Calculate Information Ratio
    information_ratio = ((Rp.mean() * 252) - (Rb.mean() * 252)) / tracking_error if tracking_error != 0 else np.nan
    
    # Calculate Omega Ratio
    excess_returns = Rp - threshold_return
    omega_ratio = (
        excess_returns[excess_returns > 0].sum() / -excess_returns[excess_returns < 0].sum()
        if excess_returns[excess_returns < 0].sum() != 0
        else np.nan
    )
    
    # Store results
    metrics = {
        "Ticker": ticker,
        "Beta": beta,
        "Alpha (Annualized)": annualized_alpha,
        "Treynor Ratio": treynor_ratio,
        "Information Ratio": information_ratio,
        "Omega Ratio": omega_ratio,
    }
    
    additional_metrics.append(metrics)

# Create DataFrame with additional performance metrics
df_additional_metrics = pd.DataFrame(additional_metrics)

# Display results
print("\nAdditional Performance Metrics:\n", df_additional_metrics)

[*********************100%***********************]  1 of 1 completed


Additional Performance Metrics:
          Ticker     Beta  Alpha (Annualized)  Treynor Ratio  \
0  TATAPOWER.NS  1.11351            0.053811       0.160694   
1  POWERGRID.NS  0.59378            0.120767       0.315755   

   Information Ratio  Omega Ratio  
0           0.220547     1.111615  
1           0.322521     1.166031  


In [83]:
import numpy as np
import pandas as pd
import yfinance as yf

# Define benchmark index
benchmark_ticker = "^NSEI"

# Fetch benchmark historical data
def fetch_benchmark_data(ticker, start="2014-01-01", end="2024-01-01"):
    benchmark_data = yf.download(ticker, start=start, end=end)

    # Check if the download was successful
    if benchmark_data.empty:
        raise ValueError(f"Failed to fetch data for {ticker}. Please check the ticker symbol.")

    # Reset index to ensure "Date" is a column
    benchmark_data.reset_index(inplace=True)

    # **FLATTEN MULTIINDEX COLUMNS**
    benchmark_data.columns = [col[0] if isinstance(col, tuple) else col for col in benchmark_data.columns]

    # Use "Adj Close" if "Close" is missing
    if "Close" not in benchmark_data.columns:
        if "Adj Close" in benchmark_data.columns:
            benchmark_data.rename(columns={"Adj Close": "Close"}, inplace=True)
        else:
            raise KeyError("Neither 'Close' nor 'Adj Close' found in benchmark data.")

    # Calculate daily returns
    benchmark_data["Returns"] = benchmark_data["Close"].pct_change()
    benchmark_data.dropna(inplace=True)

    return benchmark_data

# Fetch NSEI benchmark data
benchmark_data = fetch_benchmark_data(benchmark_ticker)
print(benchmark_data.head())  # Verify the first few rows


[*********************100%***********************]  1 of 1 completed

        Date        Close         High          Low         Open  Volume  \
1 2014-01-03  6211.149902  6221.700195  6171.250000  6194.549805  139000   
2 2014-01-06  6191.450195  6224.700195  6170.250000  6220.850098  118300   
3 2014-01-07  6162.250000  6221.500000  6144.750000  6203.899902  138600   
4 2014-01-08  6174.600098  6192.100098  6160.350098  6178.049805  146900   
5 2014-01-09  6168.350098  6188.049805  6148.250000  6181.700195  150100   

    Returns  
1 -0.001607  
2 -0.003172  
3 -0.004716  
4  0.002004  
5 -0.001012  


In [81]:
pip install pandas==1.5.3



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [84]:
import numpy as np
import pandas as pd
import yfinance as yf
import pyfolio as pf
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Define benchmark index
benchmark_ticker = "^NSEI"

def fetch_benchmark_data(ticker, start="2014-01-01", end="2024-01-01"):
    benchmark_data = yf.download(ticker, start=start, end=end)
    if benchmark_data.empty:
        raise ValueError(f"Failed to fetch data for {ticker}. Please check the ticker symbol.")
    benchmark_data.reset_index(inplace=True)
    benchmark_data.columns = [col[0] if isinstance(col, tuple) else col for col in benchmark_data.columns]
    if "Close" not in benchmark_data.columns:
        if "Adj Close" in benchmark_data.columns:
            benchmark_data.rename(columns={"Adj Close": "Close"}, inplace=True)
        else:
            raise KeyError("Neither 'Close' nor 'Adj Close' found in benchmark data.")
    benchmark_data["Benchmark_Return"] = benchmark_data["Close"].pct_change()
    return benchmark_data[["Date", "Benchmark_Return"]].dropna()

# Fetch NSEI benchmark data
benchmark_data = fetch_benchmark_data(benchmark_ticker)

# Performance metrics storage
performance_metrics = []
additional_metrics = []
risk_free_rate = 0.03 / 252  # Convert annual to daily
threshold_return = 0.0

for ticker, df in stock_dataframes.items():
    df = df.copy()
    if "Date" not in df.columns:
        df.reset_index(inplace=True)
    df.set_index("Date", inplace=True)
    if "Close" in df.columns:
        df["Returns"] = df["Close"].pct_change()
    if "Returns" not in df.columns:
        print(f"Skipping {ticker} due to missing 'Returns' column.")
        continue
    
    df.index = df.index.tz_localize(None)
    benchmark_data["Date"] = benchmark_data["Date"].dt.tz_localize(None)
    df = df.join(benchmark_data.set_index("Date"), how="inner")
    if "Benchmark_Return" not in df.columns:
        print(f"Skipping {ticker} due to missing benchmark returns.")
        continue
    
    Rp = df["Returns"]
    Rb = df["Benchmark_Return"]
    beta, alpha, _, _, _ = linregress(Rb, Rp)
    annualized_alpha = (Rp.mean() * 252) - (risk_free_rate * 252 + beta * ((Rb.mean() * 252) - (risk_free_rate * 252)))
    treynor_ratio = ((Rp.mean() * 252) - (risk_free_rate * 252)) / beta if beta != 0 else np.nan
    tracking_error = np.std(Rp - Rb) * np.sqrt(252)
    information_ratio = ((Rp.mean() * 252) - (Rb.mean() * 252)) / tracking_error if tracking_error != 0 else np.nan
    excess_returns = Rp - threshold_return
    omega_ratio = (
        excess_returns[excess_returns > 0].sum() / -excess_returns[excess_returns < 0].sum()
        if excess_returns[excess_returns < 0].sum() != 0
        else np.nan
    )
    
    perf_stats = pf.timeseries.perf_stats(Rp)
    performance_metrics.append({
        "Ticker": ticker,
        "Annual Return": perf_stats["Annual return"],
        "Sharpe Ratio": perf_stats["Sharpe ratio"],
        "Sortino Ratio": perf_stats["Sortino ratio"],
        "Max Drawdown": perf_stats["Max drawdown"],
        "Calmar Ratio": perf_stats["Calmar ratio"],
    })
    additional_metrics.append({
        "Ticker": ticker,
        "Beta": beta,
        "Alpha (Annualized)": annualized_alpha,
        "Treynor Ratio": treynor_ratio,
        "Information Ratio": information_ratio,
        "Omega Ratio": omega_ratio,
    })

# Create DataFrames
df_performance = pd.DataFrame(performance_metrics)
df_additional_metrics = pd.DataFrame(additional_metrics)

# Display results
print("\nPyfolio Performance Metrics:\n", df_performance)
print("\nAdditional Performance Metrics:\n", df_additional_metrics)


[*********************100%***********************]  1 of 1 completed


Pyfolio Performance Metrics:
          Ticker  Annual Return  Sharpe Ratio  Sortino Ratio  Max Drawdown  \
0  TATAPOWER.NS        0.15820      0.591638       0.889934     -0.746467   
1  POWERGRID.NS        0.20653      0.892332       1.345975     -0.291147   

   Calmar Ratio  
0      0.211931  
1      0.709366  

Additional Performance Metrics:
          Ticker     Beta  Alpha (Annualized)  Treynor Ratio  \
0  TATAPOWER.NS  1.11351            0.053811       0.160694   
1  POWERGRID.NS  0.59378            0.120767       0.315755   

   Information Ratio  Omega Ratio  
0           0.220547     1.111615  
1           0.322521     1.166031  



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [ ]:
# import numpy as np
# import pandas as pd
# import yfinance as yf
# import pyfolio as pf
# import matplotlib.pyplot as plt
# from scipy.stats import linregress

# # Define benchmark index
# benchmark_ticker = "^NSEI"

# # Fetch benchmark historical data
# def fetch_benchmark_data(ticker, start="2014-01-01", end="2024-01-01"):
#     benchmark_data = yf.download(ticker, start=start, end=end)
    
#     if benchmark_data.empty:
#         raise ValueError(f"Failed to fetch data for {ticker}. Please check the ticker symbol.")
    
#     benchmark_data.reset_index(inplace=True)
    
#     # Use "Adj Close" if "Close" is missing
#     if "Close" not in benchmark_data.columns:
#         if "Adj Close" in benchmark_data.columns:
#             benchmark_data.rename(columns={"Adj Close": "Close"}, inplace=True)
#         else:
#             raise KeyError("Neither 'Close' nor 'Adj Close' found in benchmark data.")
    
#     benchmark_data["Benchmark_Return"] = benchmark_data["Close"].pct_change()
#     benchmark_data = benchmark_data[["Date", "Benchmark_Return"]].dropna()
    
#     return benchmark_data

# # Fetch NSEI benchmark data
# benchmark_data = fetch_benchmark_data(benchmark_ticker)

# # Define risk-free rate and threshold return
# risk_free_rate = 0.03 / 252  # Convert annual to daily
# threshold_return = 0.0

# performance_metrics = []
# additional_metrics = []

# for ticker, df in stock_dataframes.items():
#     df = df.copy()
    
#     if "Date" not in df.columns:
#         df.reset_index(inplace=True)
    
#     df.set_index("Date", inplace=True)
    
#     if "Close" in df.columns:
#         df["Returns"] = df["Close"].pct_change()
    
#     if "Returns" not in df.columns:
#         print(f"Skipping {ticker} due to missing 'Returns' column.")
#         continue
    
#     # Convert both dataframes to timezone-naive
#     df.index = df.index.tz_localize(None)
#     benchmark_data["Date"] = benchmark_data["Date"].dt.tz_localize(None)

#     df = df.join(benchmark_data.set_index("Date"), how="inner")
    
#     if "Benchmark_Return" not in df.columns:
#         print(f"Skipping {ticker} due to missing benchmark returns.")
#         continue

#     # Define stock and benchmark returns
#     Rp = df["Returns"]
#     Rb = df["Benchmark_Return"]

#     # Compute Beta & Alpha using regression
#     beta, alpha, _, _, _ = linregress(Rb, Rp)

#     # Calculate additional performance metrics
#     annualized_alpha = (Rp.mean() * 252) - (risk_free_rate * 252 + beta * ((Rb.mean() * 252) - (risk_free_rate * 252)))
#     treynor_ratio = ((Rp.mean() * 252) - (risk_free_rate * 252)) / beta if beta != 0 else np.nan
#     tracking_error = np.std(Rp - Rb) * np.sqrt(252)
#     information_ratio = ((Rp.mean() * 252) - (Rb.mean() * 252)) / tracking_error if tracking_error != 0 else np.nan
#     excess_returns = Rp - threshold_return
#     omega_ratio = (excess_returns[excess_returns > 0].sum() / -excess_returns[excess_returns < 0].sum()
#                    if excess_returns[excess_returns < 0].sum() != 0 else np.nan)

#     additional_metrics.append({
#         "Ticker": ticker,
#         "Beta": beta,
#         "Alpha (Annualized)": annualized_alpha,
#         "Treynor Ratio": treynor_ratio,
#         "Information Ratio": information_ratio,
#         "Omega Ratio": omega_ratio,
#     })

#     # Extract Pyfolio performance metrics
#     returns_series = df["Returns"]
#     perf_stats = pf.timeseries.perf_stats(returns_series)

#     performance_metrics.append({
#         "Ticker": ticker,
#         "Annual Return": perf_stats["Annual return"],
#         "Sharpe Ratio": perf_stats["Sharpe ratio"],
#         "Sortino Ratio": perf_stats["Sortino ratio"],
#         "Max Drawdown": perf_stats["Max drawdown"],
#         "Calmar Ratio": perf_stats["Calmar ratio"],
#     })

# # Convert lists to DataFrames
# df_performance = pd.DataFrame(performance_metrics)
# df_additional_metrics = pd.DataFrame(additional_metrics)

# # Merge both DataFrames on "Ticker"
# df_final_metrics = pd.merge(df_performance, df_additional_metrics, on="Ticker", how="inner")

# # Display final output
# print("\nFinal Performance Metrics:\n", df_final_metrics)


[*********************100%***********************]  1 of 1 completed

Skipping TATAPOWER.NS due to missing benchmark returns.
Skipping POWERGRID.NS due to missing benchmark returns.



/var/folders/qz/0dlc8y692_34_5qr067vqw540000gn/T/ipykernel_45797/2668475401.py:61: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df = df.join(benchmark_data.set_index("Date"), how="inner")
/var/folders/qz/0dlc8y692_34_5qr067vqw540000gn/T/ipykernel_45797/2668475401.py:61: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df = df.join(benchmark_data.set_index("Date"), how="inner")


KeyError: 'Ticker'

In [ ]:
# import pandas as pd

# def rank_metric(value, thresholds):
#     """Assigns a rank based on predefined thresholds."""
#     for threshold, rank in thresholds:
#         if value >= threshold:
#             return rank
#     return "D"  # Default rank if none of the thresholds match

# # Ranking thresholds
# sharpe_thresholds = [(2.5, "AAA"), (2.0, "AA"), (1.5, "A"), (1.0, "BBB"), (0.5, "BB"), (0.0, "B"), (-0.5, "C")]
# sortino_thresholds = [(3.0, "AAA"), (2.5, "AA"), (2.0, "A"), (1.5, "BBB"), (1.0, "BB"), (0.5, "B"), (0.0, "C")]
# max_drawdown_thresholds = [(-0.1, "AAA"), (-0.15, "AA"), (-0.2, "A"), (-0.25, "BBB"), (-0.3, "BB"), (-0.35, "B"), (-0.4, "C")]
# calmar_thresholds = [(4.0, "AAA"), (3.0, "AA"), (2.0, "A"), (1.5, "BBB"), (1.0, "BB"), (0.5, "B"), (0.0, "C")]
# treynor_thresholds = [(0.5, "AAA"), (0.4, "AA"), (0.3, "A"), (0.2, "BBB"), (0.1, "BB"), (0.0, "B"), (-0.1, "C")]
# info_ratio_thresholds = [(1.0, "AAA"), (0.8, "AA"), (0.6, "A"), (0.4, "BBB"), (0.2, "BB"), (0.0, "B"), (-0.2, "C")]
# alpha_thresholds = [(0.05, "AAA"), (0.03, "AA"), (0.01, "A"), (0.0, "BBB"), (-0.01, "BB"), (-0.03, "B"), (-0.05, "C")]
# beta_thresholds = [(0.9, "AAA"), (0.8, "AA"), (0.7, "A"), (0.6, "BBB"), (0.5, "BB"), (0.4, "B"), (0.3, "C")]
# omega_thresholds = [(1.8, "AAA"), (1.6, "AA"), (1.4, "A"), (1.2, "BBB"), (1.1, "BB"), (1.0, "B"), (0.9, "C")]

# # Apply rankings to performance DataFrame
# df_performance["Sharpe Rank"] = df_performance["Sharpe Ratio"].apply(lambda x: rank_metric(x, sharpe_thresholds))
# df_performance["Sortino Rank"] = df_performance["Sortino Ratio"].apply(lambda x: rank_metric(x, sortino_thresholds))
# df_performance["Max Drawdown Rank"] = df_performance["Max Drawdown"].apply(lambda x: rank_metric(x, max_drawdown_thresholds))
# df_performance["Calmar Rank"] = df_performance["Calmar Ratio"].apply(lambda x: rank_metric(x, calmar_thresholds))

# df_additional_metrics["Treynor Rank"] = df_additional_metrics["Treynor Ratio"].apply(lambda x: rank_metric(x, treynor_thresholds))
# df_additional_metrics["Information Rank"] = df_additional_metrics["Information Ratio"].apply(lambda x: rank_metric(x, info_ratio_thresholds))
# df_additional_metrics["Alpha Rank"] = df_additional_metrics["Alpha (Annualized)"].apply(lambda x: rank_metric(x, alpha_thresholds))
# df_additional_metrics["Beta Rank"] = df_additional_metrics["Beta"].apply(lambda x: rank_metric(x, beta_thresholds))
# df_additional_metrics["Omega Rank"] = df_additional_metrics["Omega Ratio"].apply(lambda x: rank_metric(x, omega_thresholds))

# # Display ranked data


# print("\nRanked Performance Metrics:\n", df_performance)
# print("\nRanked Additional Performance Metrics:\n", df_additional_metrics)


KeyError: 'Treynor Ratio'

### This was working well, then I accidentally deleted something and I tried an hour to debug it ☠️, but the final output is unaffected.
This code basically showed the alphabetical mapping

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import pyfolio as pf
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Define benchmark index
benchmark_ticker = "^NSEI"

def fetch_benchmark_data(ticker, start="2014-01-01", end="2024-01-01"):
    benchmark_data = yf.download(ticker, start=start, end=end)
    if benchmark_data.empty:
        raise ValueError(f"Failed to fetch data for {ticker}. Please check the ticker symbol.")
    benchmark_data.reset_index(inplace=True)
    benchmark_data.columns = [col[0] if isinstance(col, tuple) else col for col in benchmark_data.columns]
    if "Close" not in benchmark_data.columns:
        if "Adj Close" in benchmark_data.columns:
            benchmark_data.rename(columns={"Adj Close": "Close"}, inplace=True)
        else:
            raise KeyError("Neither 'Close' nor 'Adj Close' found in benchmark data.")
    benchmark_data["Benchmark_Return"] = benchmark_data["Close"].pct_change()
    return benchmark_data[["Date", "Benchmark_Return"]].dropna()

# Fetch NSEI benchmark data
benchmark_data = fetch_benchmark_data(benchmark_ticker)

# Performance metrics storage
performance_metrics = []
additional_metrics = []
risk_free_rate = 0.03 / 252  # Convert annual to daily
threshold_return = 0.0

def assign_rating(value, thresholds):
    for threshold, rating in thresholds:
        if value >= threshold:
            return rating
    return "D"  # Default lowest rating

rating_scale = {
    "AAA": 8, "AA": 7, "A": 6, "BBB": 5, "BB": 4, "B": 3, "C": 2, "D": 1
}

# Define ranking criteria
ranking_criteria = {
    "Sharpe Ratio": [(2.5, "AAA"), (2.0, "AA"), (1.5, "A"), (1.0, "BBB"), (0.5, "BB"), (0.0, "B"), (-0.5, "C")],
    "Sortino Ratio": [(3.0, "AAA"), (2.5, "AA"), (2.0, "A"), (1.5, "BBB"), (1.0, "BB"), (0.5, "B"), (0.0, "C")],
    "Max Drawdown": [(-0.10, "AAA"), (-0.15, "AA"), (-0.20, "A"), (-0.25, "BBB"), (-0.30, "BB"), (-0.35, "B"), (-0.40, "C")],
    "Calmar Ratio": [(4.0, "AAA"), (3.0, "AA"), (2.0, "A"), (1.5, "BBB"), (1.0, "BB"), (0.5, "B"), (0.0, "C")],
    "Treynor Ratio": [(0.5, "AAA"), (0.4, "AA"), (0.3, "A"), (0.2, "BBB"), (0.1, "BB"), (0.0, "B"), (-0.1, "C")],
    "Information Ratio": [(1.0, "AAA"), (0.8, "AA"), (0.6, "A"), (0.4, "BBB"), (0.2, "BB"), (0.0, "B"), (-0.2, "C")],
    "Alpha (Annualized)": [(0.05, "AAA"), (0.03, "AA"), (0.01, "A"), (0.0, "BBB"), (-0.01, "BB"), (-0.03, "B"), (-0.05, "C")],
    "Beta": [(0.9, "AAA"), (0.8, "AA"), (0.7, "A"), (0.6, "BBB"), (0.5, "BB"), (0.4, "B"), (0.3, "C")],
    "Omega Ratio": [(1.8, "AAA"), (1.6, "AA"), (1.4, "A"), (1.2, "BBB"), (1.1, "BB"), (1.0, "B"), (0.9, "C")]
}

for ticker, df in stock_dataframes.items():
    df = df.copy()
    if "Date" not in df.columns:
        df.reset_index(inplace=True)
    df.set_index("Date", inplace=True)
    if "Close" in df.columns:
        df["Returns"] = df["Close"].pct_change()
    if "Returns" not in df.columns:
        print(f"Skipping {ticker} due to missing 'Returns' column.")
        continue
    
    df.index = df.index.tz_localize(None)
    benchmark_data["Date"] = benchmark_data["Date"].dt.tz_localize(None)
    df = df.join(benchmark_data.set_index("Date"), how="inner")
    if "Benchmark_Return" not in df.columns:
        print(f"Skipping {ticker} due to missing benchmark returns.")
        continue
    
    Rp = df["Returns"]
    Rb = df["Benchmark_Return"]
    beta, alpha, _, _, _ = linregress(Rb, Rp)
    annualized_alpha = (Rp.mean() * 252) - (risk_free_rate * 252 + beta * ((Rb.mean() * 252) - (risk_free_rate * 252)))
    treynor_ratio = ((Rp.mean() * 252) - (risk_free_rate * 252)) / beta if beta != 0 else np.nan
    tracking_error = np.std(Rp - Rb) * np.sqrt(252)
    information_ratio = ((Rp.mean() * 252) - (Rb.mean() * 252)) / tracking_error if tracking_error != 0 else np.nan
    excess_returns = Rp - threshold_return
    omega_ratio = (
        excess_returns[excess_returns > 0].sum() / -excess_returns[excess_returns < 0].sum()
        if excess_returns[excess_returns < 0].sum() != 0
        else np.nan
    )
    
    perf_stats = pf.timeseries.perf_stats(Rp)
    metrics = {
        "Sharpe Ratio": perf_stats["Sharpe ratio"],
        "Sortino Ratio": perf_stats["Sortino ratio"],
        "Max Drawdown": perf_stats["Max drawdown"],
        "Calmar Ratio": perf_stats["Calmar ratio"],
        "Treynor Ratio": treynor_ratio,
        "Information Ratio": information_ratio,
        "Alpha (Annualized)": annualized_alpha,
        "Beta": beta,
        "Omega Ratio": omega_ratio
    }
    ratings = {metric: assign_rating(value, ranking_criteria[metric]) for metric, value in metrics.items()}
    scores = {metric: rating_scale[ratings[metric]] for metric in ratings}
    
    performance_metrics.append({"Ticker": ticker, **metrics, **ratings, **scores})

# Create DataFrame
df_performance = pd.DataFrame(performance_metrics)

# Display results
print("\nPerformance Metrics with Ratings and Scores:\n", df_performance)


[*********************100%***********************]  1 of 1 completed


Performance Metrics with Ratings and Scores:
          Ticker  Sharpe Ratio  Sortino Ratio  Max Drawdown  Calmar Ratio  \
0  TATAPOWER.NS             4              3             1             2   
1  POWERGRID.NS             4              4             4             3   

   Treynor Ratio  Information Ratio  Alpha (Annualized)  Beta  Omega Ratio  
0              4                  4                   8     8            4  
1              6                  4                   8     4            4  



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [88]:
import numpy as np
import pandas as pd

# Sample performance metrics with scores for two stocks
data = {
    "Ticker": ["TATAPOWER.NS", "POWERGRID.NS"],
    "Sharpe Ratio": [4, 4],
    "Sortino Ratio": [3, 4],
    "Max Drawdown": [1, 4],
    "Calmar Ratio": [2, 3],
    "Treynor Ratio": [4, 6],
    "Information Ratio": [4, 4],
    "Alpha (Annualized)": [8, 8],
    "Beta": [8, 4],
    "Omega Ratio": [4, 4]
}

df = pd.DataFrame(data)

# Define weights for each metric
weights = {
    "Sharpe Ratio": 0.20,
    "Sortino Ratio": 0.15,
    "Max Drawdown": 0.10,
    "Calmar Ratio": 0.10,
    "Treynor Ratio": 0.10,
    "Information Ratio": 0.10,
    "Alpha (Annualized)": 0.10,
    "Beta": 0.10,
    "Omega Ratio": 0.05
}

# Calculate the weighted sum for each stock
df["Final Score"] = df[[metric for metric in weights]].mul(pd.Series(weights)).sum(axis=1)

# Display results
print(df)


         Ticker  Sharpe Ratio  Sortino Ratio  Max Drawdown  Calmar Ratio  Treynor Ratio  Information Ratio  Alpha (Annualized)  Beta  Omega Ratio  Final Score
0  TATAPOWER.NS             4              3             1             2              4                  4                   8     8            4         4.15
1  POWERGRID.NS             4              4             4             3              6                  4                   8     4            4         4.50


#### Physically inputed the values for the metrics in above code, need to change the code which can automatically extract it. As this will fail if I change the tickers.If someone can, please do it, I tried but couldn't find a way.

In [89]:
import numpy as np
import pandas as pd

# Assuming df_performance already contains scores for each metric
def calculate_weighted_scores(df_performance):
    # Define weights for each metric
    weights = {
        "Sharpe Ratio": 0.20,
        "Sortino Ratio": 0.15,
        "Max Drawdown": 0.10,
        "Calmar Ratio": 0.10,
        "Treynor Ratio": 0.10,
        "Information Ratio": 0.10,
        "Alpha (Annualized)": 0.10,
        "Beta": 0.10,
        "Omega Ratio": 0.05
    }
    
    # Extract relevant columns and apply weights
    df_performance["Final Score"] = df_performance[list(weights.keys())].mul(pd.Series(weights)).sum(axis=1)
    
    # Compute mean final score
    final_score_mean = df_performance["Final Score"].mean()
    
    # Define rating mapping
    rating_mapping = {
        (7.5, 8.0): "AAA",
        (7.0, 7.5): "AA",
        (6.0, 7.0): "A",
        (5.0, 6.0): "BBB",
        (4.0, 5.0): "BB",
        (3.0, 4.0): "B",
        (2.0, 3.0): "C",
        (0.0, 2.0): "D"
    }
    
    # Determine final rating
    final_rating = "D"  # Default to lowest rating
    for (lower, upper), rating in rating_mapping.items():
        if lower <= final_score_mean <= upper:
            final_rating = rating
            break
    
    print("Mean Final Score:", final_score_mean)
    print("Final Portfolio Rating:", final_rating)
    
    return df_performance

# Example usage:



df_performance = df
df_performance = calculate_weighted_scores(df_performance)
print(df_performance)


Mean Final Score: 4.325
Final Portfolio Rating: BB
         Ticker  Sharpe Ratio  Sortino Ratio  Max Drawdown  Calmar Ratio  Treynor Ratio  Information Ratio  Alpha (Annualized)  Beta  Omega Ratio  Final Score
0  TATAPOWER.NS             4              3             1             2              4                  4                   8     8            4         4.15
1  POWERGRID.NS             4              4             4             3              6                  4                   8     4            4         4.50


Can change the final score to calculate weighted avg instead of mean

## It ain't much, but atleast its honest work :D 